In [ ]:
import cv2
import os
import sys
import math
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import signal
from skimage import exposure
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#cd drive/'My Drive'/'Colab Notebooks'/CV/'Assignment 1'/

In [ ]:
#we will be using this function to display image later in notebook
def grayRGBImage(image):
    img_gray = cv2.imread(image,0) # 0 for grayscal 
    fig = plt.figure()
    fig.add_subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img_gray,cv2.COLOR_BGR2RGB))
    plt.title("Gray")
    img_colored = cv2.imread(image,1) # for colored Image
    fig.add_subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(img_colored,cv2.COLOR_BGR2RGB))
    plt.title("RGB")
    plt.show()

In [ ]:
#This function will take image as an argument and decompose that image in to RGB channels and display images
def imageDecompose(image):
  img_R = cv2.imread(image)
  img_G = cv2.imread(image)
  img_B = cv2.imread(image)
  img_R[:,:,0] = img_R[:,:,1] = 0
  img_G[:,:,0] = img_G[:,:,2] = 0
  img_B[:,:,1] = img_B[:,:,2] = 0
  # and we are left out with red channel
  fig = plt.figure()
  fig.add_subplot(1, 3, 1)
  plt.imshow(img_R)
  plt.title("Red Channel")
    # and we are left out with red channel
  fig.add_subplot(1, 3, 2)
  plt.imshow(img_G)
  plt.title("Green Channel")
    # and we are left out with red channel
  fig.add_subplot(1, 3, 3)
  plt.imshow(img_B)
  plt.title("Blue Channel")

In [ ]:
#For Blue pass 0, For Green pass 1, For Red pass 2 as a second parameter of rgbExclusion function
#This function will display all channels (RGB) and retun an image which exclude channel mentioned in 2nd parameter
def rgbExclusion(image,excludeChannel):
  imageDecompose(image)
  img_input = cv2.imread(image)
  bgr_image = img_input.copy()
  bgr_image[:,:,excludeChannel] = 0
  # and we are left out with red channel
  return bgr_image

In [ ]:
#This function will display image in grey scale as well as histogram of that image
def imageGreyWithHistogram(image):
    img = io.imread(image,1) #in skimage 1 is for grayscale 
    fig = plt.figure()
    fig.add_subplot(1, 2, 1)
    plt.imshow(img,cmap=plt.cm.gray)
    plt.title("Grey Image")
    fig.add_subplot(1, 2, 2)
    plt.hist(img.ravel(), bins = 256)
    plt.title("Grey Image Histogram")
    plt.show()

In [ ]:
#This function will apply histogram equalization on grey scale image and display histogram of that image
def imageHistogramEqualizer(image):
    img = io.imread(image,1) #in skimage 1 is for grayscale 
    fig = plt.figure()
    fig.add_subplot(1, 2, 1)
    image_sharpen_equalized = exposure.equalize_adapthist(img/np.max(np.abs(img)), clip_limit=0.03)
    plt.title("Histogram Equalized Image")
    plt.imshow(image_sharpen_equalized,cmap=plt.cm.gray)
    fig.add_subplot(1, 2, 2)
    plt.hist(image_sharpen_equalized.ravel(), bins = 256)
    plt.title("Histogram Equalized Image Histogram")
    plt.show()

In [ ]:
#Function for convolving kernel with image
def myConvolve2d(image, kernel):
    kernel = np.flipud(np.fliplr(kernel))    # Flip the kernel
    output = np.zeros_like(image)            # convolution output
    # Add zero padding to the input image
    image_padded = np.zeros((image.shape[0] + 4, image.shape[1] + 4))   
    image_padded[2:-2, 2:-2] = image
    
    # Loop over every pixel of the image and implement convolution operation (element wise multiplication and summation). 
    # You can use two loops. The result is stored in the variable output.
    
    for x in range(image.shape[0]):     # Loop over every pixel of the image
        for y in range(image.shape[1]):
            # element-wise multiplication and summation 
            output[x,y]=(kernel*image_padded[x:x+5,y:y+5]).sum()
        
    
    return output


In [ ]:
#To add Gussian or SnP noise depending on 1st paramter of this function
def noiseGaussian_SnP(noise,image):
   if noise == "Gaussian":
      row,col= image.shape
      mean = 0
      var = 0.1
      sigma = var**0.5
      gauss = np.random.normal(mean,sigma,(row,col))
      gauss = gauss.reshape(row,col)
      noisy = image + gauss
      return noisy
   elif noise == "SnP":
      row,col = image.shape
      s_vs_p = 0.5
      amount = 0.09
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 1

      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out

In [ ]:
#To plot 3D mesh of kernels
def meshPlot(x,y,z):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x,y,z,cmap=cm.jet)
    ax.set_title("sigma=1")
    plt.show()

In [ ]:
#grad_magnitude function require image with filters to compute derivative in x,y and magnitude
def grad_magnitude(img,sobelX,sobelY):

    #Sobel X part
    img_SobelX =  cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
    plt.rcParams['figure.figsize'] = (20.0, 16.0) # set default size of plots
    fig = plt.figure()
    fig.add_subplot(1, 4, 1)
    plt.imshow(img,cmap=plt.cm.gray)
    plt.title("GrayScale")
    fig.add_subplot(1, 4, 2)
    plt.imshow(img_SobelX,cmap=plt.cm.gray)
    plt.title("Sobel-X")

    #Sobel Y part
    img_SobelY = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    fig.add_subplot(1, 4, 3)
    plt.imshow(img_SobelY,cmap=plt.cm.gray)
    plt.title("Sobel-Y")

    #Magnitude
    mag1_Grad= np.sqrt((np.square(img_SobelX)+np.square(img_SobelY)))
    fig.add_subplot(1, 4, 4)
    plt.imshow(mag1_Grad,cmap=plt.cm.gray)
    plt.title("Magnitude")

In [ ]:
#compute_LoG will apply guassian kernel and then find LoG from then image.
def compute_LoG(img,laplacian):
    img_gaussianFilter = cv2.GaussianBlur(img, (3,3),sigmaX=3)
    img_Laplacian = cv2.Laplacian(img_gaussianFilter,cv2.CV_64F)
    plt.rcParams['figure.figsize'] = (20.0, 16.0) # set default size of plots
    fig = plt.figure()
    fig.add_subplot(1, 3, 1)
    plt.imshow(img,cmap=plt.cm.gray)
    plt.title("GrayScale")
    fig.add_subplot(1, 3, 2)
    plt.imshow(img_gaussianFilter,cmap=plt.cm.gray)
    plt.title("Gaussian")
    fig.add_subplot(1, 3, 3)
    plt.imshow(img_Laplacian,cmap=plt.cm.gray)
    plt.title("Laplacian")